In [2]:
import pandas as pd
import numpy as np

# Load the motion data
#df = pd.read_csv('/home/pshmo/summer_milestone_2-1/datasets/nfl-playing-surface-analytics/PlayerTrackData.csv')
df = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/nfl-playing-surface-analytics/PlayerTrackData.csv')

# clean injury record data
#injury_record = pd.read_csv('/home/pshmo/summer_milestone_2-1/datasets/nfl-playing-surface-analytics/InjuryRecord.csv')
injury_record = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/nfl-playing-surface-analytics/InjuryRecord.csv')

# fix missing PlayKeys and drop duplicates
#playlist = pd.read_csv('/home/pshmo/summer_milestone_2-1/datasets/nfl-playing-surface-analytics/PlayList.csv')
playlist = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/nfl-playing-surface-analytics/PlayList.csv')
last_plays = playlist.groupby('GameID')['PlayKey'].last() # get each players last play in each game
injury_record.loc[injury_record['PlayKey'].isnull(), 'PlayKey'] = injury_record['GameID'].map(last_plays)[injury_record['PlayKey'].isnull()].values  # fill in missing PlayKeys with last play of the game
injury_record = injury_record.groupby('PlayKey').first().reset_index()  # keep only the first record for each PlayKey to drop duplicates


In [3]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

def build_vectorized_motion_features(df):
    """
    Build aggregated motion features for ALL plays at once using vectorized operations
    This will be much faster than processing plays one by one
    """
    print("Starting vectorized feature calculation...")
    
    # Sort by PlayKey and time to ensure proper order
    df = df.sort_values(['PlayKey', 'time']).reset_index(drop=True)
    
    # Calculate time differences within each play
    df['time_diff'] = df.groupby('PlayKey')['time'].diff().fillna(0)
    
    # Calculate velocities from position changes (vectorized)
    df['vx'] = df.groupby('PlayKey')['x'].diff() / df['time_diff']
    df['vy'] = df.groupby('PlayKey')['y'].diff() / df['time_diff']
    df['speed_calc'] = np.sqrt(df['vx']**2 + df['vy']**2)
    
    # Use provided speed where available, calculated as backup
    df['speed'] = df['s'].fillna(df['speed_calc'])
    
    # Calculate acceleration (vectorized)
    df['ax'] = df.groupby('PlayKey')['vx'].diff() / df['time_diff']
    df['ay'] = df.groupby('PlayKey')['vy'].diff() / df['time_diff']
    df['acceleration'] = np.sqrt(df['ax']**2 + df['ay']**2)
    
    # Handle infinite values
    for col in ['vx', 'vy', 'speed_calc', 'speed', 'ax', 'ay', 'acceleration']:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Jerk rate (vectorized)
    df['jerk'] = df.groupby('PlayKey')['acceleration'].diff() / df['time_diff']
    df['jerk'] = df['jerk'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Direction changes (vectorized)
    df['dir_change'] = df.groupby('PlayKey')['dir'].diff()
    df['orient_change'] = df.groupby('PlayKey')['o'].diff()
    
    # Handle angle wraparound
    for col in ['dir_change', 'orient_change']:
        df[col] = np.where(df[col] > 180, df[col] - 360, df[col])
        df[col] = np.where(df[col] < -180, df[col] + 360, df[col])
    
    # Angular velocities (vectorized)
    df['angular_vel'] = df['dir_change'] / df['time_diff']
    df['angular_vel'] = df['angular_vel'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    print("Calculated derivatives, now aggregating features...")
    
    # Create boolean masks for thresholds (faster than applying to groups)
    df['rapid_decel'] = df['acceleration'] < -3.0
    df['rapid_accel'] = df['acceleration'] > 3.0
    df['above_avg_speed'] = df['speed'] > df.groupby('PlayKey')['speed'].transform('mean')
    df['sharp_turn'] = df['angular_vel'].abs() > 45
    
    # Calculate quantiles for each play (vectorized)
    df['speed_95th'] = df.groupby('PlayKey')['speed'].transform(lambda x: x.quantile(0.95))
    df['jerk_90th'] = df.groupby('PlayKey')['jerk'].transform(lambda x: x.abs().quantile(0.9))
    df['high_jerk'] = df['jerk'].abs() > df['jerk_90th']
    
    # Get first and last positions for displacement calculation
    first_last = df.groupby('PlayKey').agg({
        'x': ['first', 'last'],
        'y': ['first', 'last'],
        'time': ['min', 'max']
    }).round(6)
    
    first_last.columns = ['x_first', 'x_last', 'y_first', 'y_last', 'time_min', 'time_max']
    first_last['displacement'] = np.sqrt(
        (first_last['x_last'] - first_last['x_first'])**2 + 
        (first_last['y_last'] - first_last['y_first'])**2
    )
    first_last['play_duration'] = first_last['time_max'] - first_last['time_min']
    
    print("Aggregating final features...")
    
    # Aggregate all features at once (much faster)
    agg_dict = {
        # Basic aggregations
        'dis': ['sum', 'mean', 'std'],
        'speed': ['max', 'mean', 'std'],
        'acceleration': ['max', 'mean', 'std', 'min'],
        'jerk': [lambda x: x.abs().max(), lambda x: x.abs().mean(), lambda x: x.abs().quantile(0.95)],
        'dir_change': [lambda x: x.abs().sum()],
        'angular_vel': [lambda x: x.abs().max(), lambda x: x.abs().mean()],
        'x': [lambda x: x.max() - x.min()],
        'y': [lambda x: x.max() - x.min()],
        'o': ['std'],
        'dir': ['std'],
        
        # Boolean aggregations (counts)
        'rapid_decel': 'sum',
        'rapid_accel': 'sum',
        'above_avg_speed': ['sum', 'count'],  # sum for count above avg, count for total
        'sharp_turn': 'sum',
        'high_jerk': 'sum',
        
        # For unique values
        'speed_95th': 'first',  # These are constant within each play
    }
    
    features = df.groupby('PlayKey').agg(agg_dict).round(6)
    
    # Flatten column names
    features.columns = ['_'.join(col) if col[1] else col[0] for col in features.columns]
    
    # Rename columns to match original naming
    rename_dict = {
        'dis_sum': 'total_distance',
        'dis_mean': 'avg_distance_per_frame', 
        'dis_std': 'distance_std',
        'speed_max': 'max_speed',
        'speed_mean': 'avg_speed',
        'speed_std': 'speed_std',
        'acceleration_max': 'max_acceleration',
        'acceleration_mean': 'avg_acceleration',
        'acceleration_std': 'acceleration_std',
        'acceleration_min': 'min_acceleration',
        'jerk_<lambda_0>': 'max_jerk',
        'jerk_<lambda_1>': 'avg_jerk',
        'jerk_<lambda_2>': 'jerk_95th',
        'dir_change_<lambda>': 'total_direction_change',
        'angular_vel_<lambda_0>': 'max_angular_velocity',
        'angular_vel_<lambda_1>': 'avg_angular_velocity',
        'x_<lambda>': 'x_range',
        'y_<lambda>': 'y_range',
        'o_std': 'orientation_std',
        'dir_std': 'direction_std',
        'rapid_decel_sum': 'rapid_decelerations',
        'rapid_accel_sum': 'rapid_accelerations',
        'above_avg_speed_sum': 'time_above_avg_speed_count',
        'above_avg_speed_count': 'total_frames',
        'sharp_turn_sum': 'sharp_turns',
        'high_jerk_sum': 'high_jerk_events',
        'speed_95th_first': 'speed_95th',
    }
    
    features = features.rename(columns=rename_dict)
    
    # Add back the first/last position features
    features = features.join(first_last[['displacement', 'play_duration']])
    
    # Calculate derived features
    features['max_deceleration'] = np.maximum(0, -features['min_acceleration'])
    features['field_coverage'] = features['x_range'] * features['y_range']
    features['time_above_avg_speed'] = features['time_above_avg_speed_count'] / features['total_frames']
    features['movement_efficiency'] = np.where(
        features['total_distance'] > 0,
        features['displacement'] / features['total_distance'],
        0
    )
    features['speed_cv'] = features['speed_std'] / (features['avg_speed'] + 1e-6)
    features['acceleration_cv'] = features['acceleration_std'] / (features['avg_acceleration'] + 1e-6)
    
    # Clean up temporary columns
    features = features.drop(['min_acceleration', 'time_above_avg_speed_count', 'total_frames'], axis=1)
    
    # Handle any remaining NaN values
    features = features.fillna(0)
    
    print(f"Completed feature calculation for {len(features)} plays")
    return features

def process_motion_chunk_v2(chunk_df):
    """
    Your existing motion chunk processing function (unchanged but optimized)
    """
    # Make an explicit copy to avoid warnings
    chunk_df = chunk_df.copy()
    
    # Time is already in seconds, just use directly as relative_time
    chunk_df['relative_time'] = chunk_df['time']
    
    # Create time deciles based on relative time
    chunk_df['time_decile'] = chunk_df.groupby(['PlayKey'])['relative_time'].transform(
        lambda x: pd.cut(x, bins=10, labels=[f'slice_{i}' for i in range(10)], duplicates='drop')
    )
    
    # Pivot to wide - using more metrics now
    motion_wide = chunk_df.pivot_table(
        index=['PlayKey'], 
        columns='time_decile',
        values=['dis', 'x', 'y', 'o', 'dir', 's'],
        aggfunc='mean',
        observed=True
    )
    
    # Flatten columns
    motion_wide.columns = [f'{metric}_{time}' for metric, time in motion_wide.columns]
    
    # Fill missing values by interpolating across time slices
    for metric in ['dis', 'x', 'y', 'o', 'dir', 's']:
        metric_cols = [col for col in motion_wide.columns if col.startswith(f'{metric}_slice_')]
        if metric_cols:
            motion_wide[metric_cols] = motion_wide[metric_cols].interpolate(axis=1, method='linear')
    
    return motion_wide

def process_playertrack_data_fast(df, chunk_size=5000):
    """
    Faster version of your playertrack processing
    """
    motion_wide_list = []
    unique_plays = df['PlayKey'].unique()
    
    print(f"Processing {len(unique_plays)} plays in chunks of {chunk_size}")
    
    for i in range(0, len(unique_plays), chunk_size):
        play_chunk = unique_plays[i:i+chunk_size]
        chunk_df = df[df['PlayKey'].isin(play_chunk)]
        
        motion_wide_chunk = process_motion_chunk_v2(chunk_df)
        motion_wide_list.append(motion_wide_chunk)
        
        print(f"Processed {min(i+chunk_size, len(unique_plays))} of {len(unique_plays)} plays")
    
    # Combine all chunks
    motion_wide = pd.concat(motion_wide_list, axis=0)
    return motion_wide

# Usage examples:
# 
# # For aggregated features (MUCH faster now):
#motion_features = build_vectorized_motion_features(df)
# 
# # For time-series features (optimized):
motion_wide = process_playertrack_data_fast(df, chunk_size=5000)
# 
# # Combine both:
# combined_features = motion_wide.join(motion_features, how='inner')

Processing 266960 plays in chunks of 5000
Processed 5000 of 266960 plays
Processed 10000 of 266960 plays
Processed 15000 of 266960 plays
Processed 20000 of 266960 plays
Processed 25000 of 266960 plays
Processed 30000 of 266960 plays
Processed 35000 of 266960 plays
Processed 40000 of 266960 plays
Processed 45000 of 266960 plays
Processed 50000 of 266960 plays
Processed 55000 of 266960 plays
Processed 60000 of 266960 plays
Processed 65000 of 266960 plays
Processed 70000 of 266960 plays
Processed 75000 of 266960 plays
Processed 80000 of 266960 plays
Processed 85000 of 266960 plays
Processed 90000 of 266960 plays
Processed 95000 of 266960 plays
Processed 100000 of 266960 plays
Processed 105000 of 266960 plays
Processed 110000 of 266960 plays
Processed 115000 of 266960 plays
Processed 120000 of 266960 plays
Processed 125000 of 266960 plays
Processed 130000 of 266960 plays
Processed 135000 of 266960 plays
Processed 140000 of 266960 plays
Processed 145000 of 266960 plays
Processed 150000 of 2

In [16]:
#df = pd.read_csv('/home/pshmo/summer_milestone_2-1/datasets/nfl-playing-surface-analytics/PlayList.csv')  # Load your data here
df = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/nfl-playing-surface-analytics/PlayList.csv')  # Load your data here
playlist_short = df.copy()
playlist_short = playlist_short[['PlayerKey', 'GameID', 'PlayKey', 'RosterPosition', 'PlayerDay', 'PlayerGame', 'FieldType', 'PlayType']]
df_final = motion_wide.merge(playlist_short, on='PlayKey', how='left')

In [17]:
df_final.head()

,PlayKey,dir_slice_0,dir_slice_1,dir_slice_2,dir_slice_3,dir_slice_4,dir_slice_5,dir_slice_6,dir_slice_7,dir_slice_8,...,y_slice_7,y_slice_8,y_slice_9,PlayerKey,GameID,RosterPosition,PlayerDay,PlayerGame,FieldType,PlayType
0,26624-1-1,191.485333,224.039667,276.865000,270.013667,233.042667,102.270000,116.333667,173.968000,192.327000,...,27.708000,24.982667,23.769667,26624,26624-1,Quarterback,1,1,Synthetic,Pass
1,26624-1-10,321.335000,223.575357,258.757407,289.139286,113.188148,50.242143,101.534444,217.098929,199.318519,...,26.300000,24.490000,22.715714,26624,26624-1,Quarterback,1,1,Synthetic,Pass
2,26624-1-11,260.440270,137.356757,291.104595,188.302973,88.250270,128.555556,262.251892,300.738919,210.378919,...,26.081622,27.830541,26.090270,26624,26624-1,Quarterback,1,1,Synthetic,Rush
3,26624-1-12,110.672778,110.170833,200.163429,66.518611,216.207222,248.813429,117.101944,214.901429,233.532778,...,24.062857,22.196667,21.098333,26624,26624-1,Quarterback,1,1,Synthetic,Pass
4,26624-1-13,125.639231,232.700000,38.018000,184.952692,181.747600,199.459200,155.309231,149.863077,232.201600,...,23.850769,22.677600,21.692308,26624,26624-1,Quarterback,1,1,Synthetic,Pass


In [18]:
df_final['PlayType'] = df_final['PlayType'].fillna('Unknown') 
print(df_final.isna().sum())  # Check for any remaining NaNs from the first merge

PlayKey           0
dir_slice_0       0
dir_slice_1       0
dir_slice_2       0
dir_slice_3       0
                 ..
RosterPosition    0
PlayerDay         0
PlayerGame        0
FieldType         0
PlayType          0
Length: 68, dtype: int64


In [19]:
# merge target
df_final = df_final.merge(injury_record, on='PlayKey', how='left')
df_final = df_final.drop(columns=['PlayerKey_x', 'GameID_x', 'PlayerKey_y', 'GameID_y', 'Surface'])  # Drop redundant columns

In [20]:
df_final

,PlayKey,dir_slice_0,dir_slice_1,dir_slice_2,dir_slice_3,dir_slice_4,dir_slice_5,dir_slice_6,dir_slice_7,dir_slice_8,...,RosterPosition,PlayerDay,PlayerGame,FieldType,PlayType,BodyPart,DM_M1,DM_M7,DM_M28,DM_M42
0,26624-1-1,191.485333,224.039667,276.865000,270.013667,233.042667,102.270000,116.333667,173.968000,192.327000,...,Quarterback,1,1,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
1,26624-1-10,321.335000,223.575357,258.757407,289.139286,113.188148,50.242143,101.534444,217.098929,199.318519,...,Quarterback,1,1,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
2,26624-1-11,260.440270,137.356757,291.104595,188.302973,88.250270,128.555556,262.251892,300.738919,210.378919,...,Quarterback,1,1,Synthetic,Rush,NaN,NaN,NaN,NaN,NaN
3,26624-1-12,110.672778,110.170833,200.163429,66.518611,216.207222,248.813429,117.101944,214.901429,233.532778,...,Quarterback,1,1,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
4,26624-1-13,125.639231,232.700000,38.018000,184.952692,181.747600,199.459200,155.309231,149.863077,232.201600,...,Quarterback,1,1,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266955,47888-9-54,261.087826,122.009565,137.890455,114.269565,161.362174,310.110909,330.274783,167.346364,167.724783,...,Cornerback,71,9,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
266956,47888-9-6,258.117949,154.227949,207.245526,219.458462,294.992051,82.230000,19.501026,202.865263,205.459231,...,Cornerback,71,9,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN
266957,47888-9-7,171.075882,149.099394,114.850606,203.711765,153.004242,225.313636,148.673529,123.513030,163.729394,...,Cornerback,71,9,Synthetic,Rush,NaN,NaN,NaN,NaN,NaN
266958,47888-9-8,343.545937,158.072258,137.502581,338.527097,244.781290,159.303871,51.431613,87.599355,186.227419,...,Cornerback,71,9,Synthetic,Pass,NaN,NaN,NaN,NaN,NaN


In [21]:
injury_cols = [['DM_M1', 'DM_M7', 'DM_M28', 'DM_M42']]
for col in injury_cols[0]:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0)  # Fill NaNs with 0 for injury columns

In [22]:
# create binary target 
df_final['injury'] = df_final['DM_M1']
df_final['BodyPart'] = df_final['BodyPart'].fillna('Non-Injury')  # Fill NaNs in BodyPart with 'Non-Injury'
# Drop the 'Surface' column as it is not needed
one_hot_cols = ['RosterPosition', 'FieldType', 'PlayType', 'BodyPart']
df_final = pd.get_dummies(df_final, columns=one_hot_cols, drop_first=True)  # One-hot encode categorical variables
# Convert all boolean columns in df_final to 0/1 integers
bool_cols = df_final.select_dtypes(include='bool').columns
df_final[bool_cols] = df_final[bool_cols].astype(int)
df_final.head()  # Display the final processed DataFrame

,PlayKey,dir_slice_0,dir_slice_1,dir_slice_2,dir_slice_3,dir_slice_4,dir_slice_5,dir_slice_6,dir_slice_7,dir_slice_8,...,PlayType_Punt,PlayType_Punt Not Returned,PlayType_Punt Returned,PlayType_Rush,PlayType_Unknown,BodyPart_Foot,BodyPart_Heel,BodyPart_Knee,BodyPart_Non-Injury,BodyPart_Toes
0,26624-1-1,191.485333,224.039667,276.865000,270.013667,233.042667,102.270000,116.333667,173.968000,192.327000,...,0,0,0,0,0,0,0,0,1,0
1,26624-1-10,321.335000,223.575357,258.757407,289.139286,113.188148,50.242143,101.534444,217.098929,199.318519,...,0,0,0,0,0,0,0,0,1,0
2,26624-1-11,260.440270,137.356757,291.104595,188.302973,88.250270,128.555556,262.251892,300.738919,210.378919,...,0,0,0,1,0,0,0,0,1,0
3,26624-1-12,110.672778,110.170833,200.163429,66.518611,216.207222,248.813429,117.101944,214.901429,233.532778,...,0,0,0,0,0,0,0,0,1,0
4,26624-1-13,125.639231,232.700000,38.018000,184.952692,181.747600,199.459200,155.309231,149.863077,232.201600,...,0,0,0,0,0,0,0,0,1,0


In [23]:
df_final.to_csv('processed_motion_features.csv', index=False)